# Tractography (TCK, TRK, TRX, VTK)

### Installation
This notebook can be run on a local server or on a virtualized environement like Google Colab.

In [ ]:
#run if using Google Colab or not building from source
try:
    # -q silence the output from pip. Remove this flag if you need 
    # to debug an installation issue.
    %pip install -q ipyniivue
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

### Imports

In [ ]:
from ipyniivue import Niivue
import ipywidgets as widgets

### Display Controls
interactive controls to customize visualization

In [ ]:
# Fiber Length Slider
length_slider = widgets.IntSlider(min=1, max=80, value=3, description="Length")
def on_length_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberLength", change["new"])
length_slider.observe(on_length_change, names="value")

# Fiber Dither Slider
dither_slider = widgets.FloatSlider(min=0, max=10, value=1, description="Dither")
def on_dither_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberDither", change["new"] * 0.1)
dither_slider.observe(on_dither_change, names="value")

# Fiber Color Dropdown
color_dropdown = widgets.Dropdown(
    options=[
        ("Global direction", "Global"),
        ("Local direction", "Local"),
        ("Fixed", "Fixed"),
        ("First Per Vertex Type (if available)", "DPV0"),
        ("First Per Streamline Type (if available)", "DPS0"),
    ],
    value="Global",
    description="Fiber color:",
)

def on_color_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberColor", change["new"])
color_dropdown.observe(on_color_change, names="value")

# Fiber Decimation Dropdown
decimation_dropdown = widgets.Dropdown(
    options=[
        ("100%", 1),
        ("50%", 2),
        ("25%", 4),
        ("10%", 10),
    ],
    value=1,
    description="Fiber reduction:",
)

def on_decimation_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberDecimationStride", change["new"])
decimation_dropdown.observe(on_decimation_change, names="value")

# Arrange controls vertically
controls = widgets.VBox([length_slider, dither_slider, color_dropdown, decimation_dropdown])

### Niivue

In [ ]:
# Creating a NiiVue object
nv = Niivue(back_color=[0.8, 0.8, 1, 1], show_3D_crosshair=True)

# Display it so it has a canvas an WebGL context. Also display the controls
display(widgets.VBox([controls, nv]))

### Load a volume and set slice type

In [ ]:
nv.add_volume("https://niivue.github.io/niivue/images/mni152.nii.gz")
nv.set_slice_type(nv.slice_type.render)

### Load a mesh

In [ ]:
nv.load_meshes([{"url": "https://niivue.github.io/niivue/images/dpsv.trx", "rgba255": [0, 142, 0, 255]}])

The following is in a separate cell because we have to wait for the typescript end of the widget to update .meshes variable in the python end. 

In [ ]:
nv.set_mesh_property(nv.meshes[0].id, "colormap", "blue")
nv.set_clip_plane([-0.1, 270, 0])